# HuggingFace Transformers `Pipeline`을 활용한 **Zero-shot Classification**

자연어 처리 태스크를 수행하기 위해 많은 학습 데이터가 있으면 좋은 것은 사실입니다. 하지만 실제 환경에서 다양한 라벨링 데이터를 확보하는 것은 어려운 일이며, [GPT-3](https://arxiv.org/abs/2005.14165)는 거대 규모로 학습된 언어 모델은 라벨링 데이터를 (거의) 사용하지 않고도 태스크를 수행할 수 있음을 보여주었습니다.

그렇다면 Few or Zero-shot 예측은 *GPT-3* 와 같은 모델을 활용해야만 가능한 작업일까요? 얼마만큼의 성능을 원하는지에 따라 다를 수 있겠지만, 결론은 "꼭 그렇지는 않다"입니다.

**Natural Language Inference** (이하 NLI)는 두 문장이 논리적으로 수반될 수 있는지 (`entail`), 상충하는지 (`contradict`) 혹은 논리적 관계가 딱히 존재하지 않는지 (`neutral`)를 판단하는 태스크입니다. 우리는 이러한 NLI 데이터로 학습된 모델을 활용해 *Zero-shot Classification* 을 실험해볼 수도 있습니다.

19년 공개된 [논문](https://arxiv.org/abs/1909.00161)에서는 아래와 같은 분류 방법을 제안합니다. 먼저 분류하고자 하는 문장을 `premise`에 작성합니다. 그리고 문장이 특정 클래스에 속하는지 알아보기 위해 `hypothesis`를 **이 문장은 {class}에 관한 것이다.**와 같이 작성해줍니다.

```
premise = "Who are you voting for in 2020?"
hypothesis = "This text is about politics."
pair = f"{premise} {sep_token} {hypothesis}"
```

위 양식처럼 다양한 라벨에 따른 문장 페어를 작성할 수 있겠습니다. 작성된 `pair` 문장은 **NLI** 데이터로 학습된 모델에 입력되게 되고, 모델은 문장이 얼마나 `entail` 하는지에 대한 값을 반환해주게 됩니다. 즉, 모델은 입력된 문장이 특정 클래스에 속한다는 주장이 얼마나 논리적으로 부합하는지를 예측한다고 생각하시면 됩니다.

실제로 이처럼 두 문장의 `entail`을 예측하도록 구현된 파이프라인을 활용하면 아래와 같이 여러 클래스에 대해 학습 데이터 없이 **NLI** 모델로 하여금 *Zero-shot Classification* 을 수행하도록 할 수 있게 됩니다.

![](https://joeddav.github.io/blog/images/zsl/zsl-demo-screenshot.png)

*Zero-shot Classification* 과 관련한 더 자세한 실험 내용은 [이곳](https://joeddav.github.io/blog/2020/05/29/ZSL.html)에서 확인하실 수 있습니다.

이제 본격적으로 실험을 진행해보도록 하겠습니다. 본 노트북을 위해서는 `transformers`, `datasets` 라이브러리의 설치가 필요합니다.

In [ ]:
# !pip install transformers datasets

노트북을 실행하는데 필요한 라이브러리들을 모두 임포트합니다.

In [ ]:
import random
from abc import ABC, abstractmethod
from IPython.display import display, HTML

import pandas as pd
from tqdm import tqdm
from datasets import load_dataset, ClassLabel
from transformers import pipeline, AutoTokenizer

어떤 문장을 가지고 *Zero-shot Classification* 을 실험해보는 것이 좋을까요?

임의의 문장과 라벨을 선정해 자유롭게 실험을 해볼 수도 있겠지만, 본 노트북에서는 실제로 *Zero-shot Classificaion* 파이프라인이 어느 정도 성능을 보일 수 있는지 정량적으로 평가해보고자 합니다.

먼저, 라벨 정보가 있는 문장 분류 데이터가 필요하겠죠. 이를 위해 HuggingFace datasets 라이브러리에 등록된 KLUE 데이터셋 중, TC 데이터를 내려받습니다.

In [ ]:
datasets = load_dataset("klue", "ynat")

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/ynat/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90. Subsequent calls will reuse this data.


데이터셋을 전반적으로 살펴보기 위한 시각화 함수를 다음과 같이 정의합니다.

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."

    picks = []
    
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)

        # 이미 등록된 예제가 뽑힌 경우, 다시 추출
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)

        picks.append(pick)

    # 임의로 추출된 인덱스들로 구성된 데이터 프레임 선언
    df = pd.DataFrame(dataset[picks])

    for column, typ in dataset.features.items():
        # 라벨 클래스를 스트링으로 변환
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])

    display(HTML(df.to_html()))

앞서 정의한 함수를 활용해 훈련 데이터를 살펴보도록 합시다.

이처럼 데이터를 살펴보는 것의 장점으로는 각 라벨에 어떠한 문장들이 해당하는지에 대한 감을 익힐 수 있다는데에 있습니다.

**KLUE TC**는 정치, 경제, 세계, 스포츠, 생활문화, IT과학 그리고 사회 등 총 7개의 라벨을 지니는 데이터셋입니다.

In [ ]:
show_random_elements(datasets["validation"])

,date,guid,label,title,url
0,2020.03.19. 오전 9:05,ynat-v1_dev_07753,경제,1보 코스피 반등 출발…장중 1620선 회복,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=001&aid=0011483017
1,2017.08.21. 오전 11:35,ynat-v1_dev_00817,생활문화,필묵으로 승화한 논어…사체四體 6만4천자 9년만에 완성,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=242&oid=001&aid=0009487092
2,2018.07.18 18:19,ynat-v1_dev_01527,스포츠,올해도 반전 꿈꾸는 롯데…작년 이맘때와 성적 비슷하네요,https://sports.news.naver.com/news.nhn?oid=001&aid=0010218607
3,2018.02.09. 오전 11:00,ynat-v1_dev_03602,생활문화,주말 N 여행 호남권 간간 쫄깃쫄깃 알큰 배릿한 맛…벌교 꼬막이 왔어요,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=237&oid=001&aid=0009874310
4,2020.04.27. 오전 6:15,ynat-v1_dev_02020,경제,오락가락 규제에 지친 국내 인터넷은행 1호 사원 떠난다,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=263&oid=001&aid=0011572643
5,2019.02.11. 오후 7:37,ynat-v1_dev_08964,세계,아자디 광장에서 열린 이란 이슬람혁명 40주년 대규모 집회,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=104&sid2=234&oid=001&aid=0010627894
6,2016.09.23. 오전 9:38,ynat-v1_dev_02558,사회,정부 기초연구사업 연구비 제때 준 적 한번도 없어,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=228&oid=001&aid=0008702015
7,2018.04.26. 오전 7:01,ynat-v1_dev_06826,정치,정상회담 D1 강직 文대통령 파격 金위원장 케미 주목,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0010047601
8,2020.04.07. 오전 10:21,ynat-v1_dev_03040,사회,충주시 외국인 근로자 자가격리 이행 특별 점검,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=251&oid=001&aid=0011528501
9,2016.07.29. 오전 11:00,ynat-v1_dev_04169,생활문화,주말 N 여행 아무도 몰라야 진짜 휴가랍니다… 비밀의 계곡 3選,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=237&oid=001&aid=0008573829


이제 실험에 필요한 토크나이저를 로드합니다.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Huffon/klue-roberta-base-nli")

앞서 *Zero-shot Classification* 은 두 문장이 모델의 입력 값으로 함께 들어가므로, `sep_token`이 무엇인지 알아두도록 합니다.

In [ ]:
tokenizer.sep_token

'[SEP]'

`transformers==4.7.0` 기준 **RoBERTa**와 같이 `token_type_ids`를 사용하지 않는 모델의 경우, `pipeline`을 바로 적용하기 어렵게 [소스 코드](https://huggingface.co/transformers/_modules/transformers/pipelines/zero_shot_classification.html#ZeroShotClassificationPipeline)가 작성이 되어 있습니다.

토크나이저 객체가 두 문장이 함께 들어오면 자동으로 두 번째 문장의 `token_type_ids`를 `1`로 채워버리기 때문인데요. 이를 방지하기 위해 두 문장을 토크나이저의 입력 값으로 넣어주는게 아닌 사전에 두 문장을 직접 하나의 문장으로 이어주는 작업을 아래와 같이 거치도록 합니다.

수정부에 따라 입력된 두 문장은 `{sent1} {sep_token} {sent2}`와 같은 하나의 문장 형태로 토크나이저에 입력되게 됩니다.

In [ ]:
class ArgumentHandler(ABC):
    """
    Base interface for handling arguments for each :class:`~transformers.pipelines.Pipeline`.
    """

    @abstractmethod
    def __call__(self, *args, **kwargs):
        raise NotImplementedError()


class CustomZeroShotClassificationArgumentHandler(ArgumentHandler):
    """
    Handles arguments for zero-shot for text classification by turning each possible label into an NLI
    premise/hypothesis pair.
    """

    def _parse_labels(self, labels):
        if isinstance(labels, str):
            labels = [label.strip() for label in labels.split(",")]
        return labels

    def __call__(self, sequences, labels, hypothesis_template):
        if len(labels) == 0 or len(sequences) == 0:
            raise ValueError("You must include at least one label and at least one sequence.")
        if hypothesis_template.format(labels[0]) == hypothesis_template:
            raise ValueError(
                (
                    'The provided hypothesis_template "{}" was not able to be formatted with the target labels. '
                    "Make sure the passed template includes formatting syntax such as {{}} where the label should go."
                ).format(hypothesis_template)
            )

        if isinstance(sequences, str):
            sequences = [sequences]
        labels = self._parse_labels(labels)

        sequence_pairs = []
        for sequence in sequences:
            # 수정부: 두 문장을 페어로 입력했을 때, `token_type_ids`가 자동으로 붙는 문제를 방지하기 위해 미리 두 문장을 `sep_token` 기준으로 이어주도록 함
            sequence_pairs.extend(f"{sequence} {tokenizer.sep_token} {hypothesis_template.format(label)}" for label in labels)

        return sequence_pairs

이제 앞서 정의한 커스텀 인자 핸들러와 **KLUE NLI** 데이터를 통해 학습된 모델을 통해 분류기를 초기화해주도록 합니다.

In [ ]:
classifier = pipeline(
    "zero-shot-classification",
    args_parser=CustomZeroShotClassificationArgumentHandler(),
    model="Huffon/klue-roberta-base-nli",
)

분류기는 *분류를 원하는 문장*, *분류될 수 있는 라벨 리스트* 그리고 두 번째 문장을 만들기 위한 *템플릿* 을 받도록 설계되어 있습니다.

아래 예제에서는,

```
PC 단축키 한번이면 중계 OK…게임방송 보편화 날개 달다 {sep_token} 이는 정치에 관한 것이다.
PC 단축키 한번이면 중계 OK…게임방송 보편화 날개 달다 {sep_token} 이는 경제에 관한 것이다.
PC 단축키 한번이면 중계 OK…게임방송 보편화 날개 달다 {sep_token} 이는 세계에 관한 것이다.
PC 단축키 한번이면 중계 OK…게임방송 보편화 날개 달다 {sep_token} 이는 스포츠에 관한 것이다.
PC 단축키 한번이면 중계 OK…게임방송 보편화 날개 달다 {sep_token} 이는 생활문화에 관한 것이다.
PC 단축키 한번이면 중계 OK…게임방송 보편화 날개 달다 {sep_token} 이는 IT과학에 관한 것이다.
PC 단축키 한번이면 중계 OK…게임방송 보편화 날개 달다 {sep_token} 이는 사회에 관한 것이다.
```

의 문장 쌍들이 각각 얼마나 `entail` 한지를 계산하는 과정을 통해 클래스에 대한 예측 값이 구해지게 됩니다.

In [ ]:
sequence = "PC 단축키 한번이면 중계 OK…게임방송 보편화 날개 달다"

candidate_labels = ["정치", "경제", "세계", "스포츠", "생활문화", "IT과학", "사회"]
classifier(
    sequence,
    candidate_labels,
    hypothesis_template="이는 {}에 관한 것이다.",
)

{'labels': ['세계', 'IT과학', '스포츠', '사회', '생활문화', '경제', '정치'],
 'scores': [0.24485424160957336,
  0.23497962951660156,
  0.19247150421142578,
  0.1723412424325943,
  0.09976600110530853,
  0.02977937087416649,
  0.025807952508330345],
 'sequence': 'PC 단축키 한번이면 중계 OK…게임방송 보편화 날개 달다'}

입력으로 넣어준 문장은 사실 `IT과학`에 속하는 기사 제목이었지만, 아쉽게도 `세계` 라벨을 가장 높은 값으로 예측했군요.

하지만 `IT과학`이 그 뒤를 이어 위치한 점은 고무적입니다.

이제 **KLUE TC** 검증 데이터를 활용해 *Zero-shot Classification* 파이프라인이 얼마나 괜찮은 예측을 하는지 실험을 할 차례입니다.

먼저 아래와 같이 검증 데이터가 가지고 있는 전체 라벨 리스트를 변수로 정의해두겠습니다.

In [ ]:
id_2_label = datasets["validation"].features["label"].names
id_2_label

['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']

검증 데이터 예제를 순회하며 가장 높은 확률로 예측한 라벨이 실제 골드 라벨과 일치하는 케이스가 얼마나 발생하는지를 카운트합니다.

In [ ]:
hit = 0

for example in tqdm(datasets["validation"]):
    result = classifier(
        example["title"],
        candidate_labels,
        hypothesis_template="이는 {}에 관한 것이다.",
    )
    pred = result["labels"][0]
    gold = id_2_label[example["label"]]

    if pred == gold:
        hit += 1

100%|██████████| 9107/9107 [1:50:48<00:00,  1.37it/s]


전체 9,107개의 예제 중 4,194개 예제에 대해 올바른 예측을 수행했네요 !

In [ ]:
hit, len(datasets["validation"])

(4194, 9107)

이는 대략 **46%**의 정확도에 해당합니다. 7개 클래스를 랜덤하게 예측하는 경우, 대략 **14%**의 성능을 기록하게 될테니 훨씬 더 좋은 스타트임을 알 수 있습니다.

( \* 위 결과는 *템플릿* 양식을 변경하는 것만으로 바뀔 수 있습니다.)

In [ ]:
hit / len(datasets["validation"]) * 100

46.05248709783683

**46%**가 엄청나게 높은 점수는 아니지만 훈련 데이터가 없는 환경에서 실험을 진행하는 경우 혹은 나이브한 베이스라인을 설정할 필요가 있는 경우, 간단하게 위 파이프라인을 적용해볼 수 있을 것 같습니다.

지금까지 `transformers` 내 `pipeline`을 통해 학습 데이터 없이 문장을 분류하는 *Zero-shot Classification* 파이프라인을 알아보았습니다.

본 노트북을 통해 습득한 지식이 여러분의 업무와 학습에 도움이 되었으면 좋겠습니다.

```
허 훈 (huffonism@gmail.com)
```